# Task 1.2 - Two-dimensional functions

In [1]:
import os
import tensorflow as tf
import numpy as np
import datetime
now = datetime.datetime.now

from keras import losses, optimizers
from typing import Literal
from bokeh.io import output_notebook, show
from bokeh.palettes import Sunset10
output_notebook()

from models import CustomFFNN, ICNN
from data import get_f1, get_f2__delf2_delx
from plots import plot_loss_history, plot_x_y_z
from bokeh_saving import save_figures_button
from plot_utils import get_figure_size

PLOTS_DIR = os.path.abspath('plot_pdfs')
if not os.path.exists(PLOTS_DIR):
    os.mkdir(PLOTS_DIR)

FIG_SIZE = get_figure_size(ratio=1.3)
USE_LATEX_STYLE = True

2024-10-31 20:08:25.487630: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-31 20:08:25.492052: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-31 20:08:25.502432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 20:08:25.522848: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 20:08:25.527544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 20:08:25.541684: I tensorflow/core/platform/cpu_feature_gu

Loading BokehJS ...

## 1.2.2 - Trainable custom layer

In [2]:
x, y, f1 = get_f1()
_, _, f2, del_f2__del_x = get_f2__delf2_delx()

In [3]:
print(x.shape, y.shape, f2.shape, del_f2__del_x.shape)

(400, 1) (400, 1) (400, 1) (400, 2)


In [4]:
EPOCHS = 500

# FFNN f1
ffnn_f1 = CustomFFNN(2, [16, 16, 16, 1], ['tanh', 'tanh', 'tanh', 'linear'])
ffnn_f1.compile('adam', 'mse')
ffnn_f1.optimizer.learning_rate.assign(0.01)

t1 = now()
h = ffnn_f1.fit(tf.concat([x,y], axis=-1), f1, epochs=EPOCHS, verbose=0)
t2 = now()
print(f'FFNN f1 took', t2 - t1, '(sec) to calibrate the model')
h_ffnn_f1 = h.history['loss']


# ICNN f1
icnn_f1 = ICNN(2, [16, 16, 16, 1], ['softplus', 'softplus', 'softplus', 'linear'])
icnn_f1.compile('adam', 'mse')
icnn_f1.optimizer.learning_rate.assign(0.01)

t1 = now()
h = icnn_f1.fit(tf.concat([x,y], axis=-1), f1, epochs=EPOCHS, verbose=0)
t2 = now()
print(f'ICNN f1 took', t2 - t1, '(sec) to calibrate the model')
h_icnn_f1 = h.history['loss']


# FFNN f2
ffnn_f2 = CustomFFNN(2, [16, 16, 16, 1], ['tanh', 'tanh', 'tanh', 'linear'])
ffnn_f2.compile('adam', 'mse')
ffnn_f2.optimizer.learning_rate.assign(0.01)

t1 = now()
h = ffnn_f2.fit(tf.concat([x,y], axis=-1), f2, epochs=EPOCHS, verbose=0)
t2 = now()
print(f'FFNN f2 took', t2 - t1, '(sec) to calibrate the model')
h_ffnn_f2 = h.history['loss']


# ICNN f2
icnn_f2 = ICNN(2, [16, 16, 16, 1], ['softplus', 'softplus', 'softplus', 'linear'])
icnn_f2.compile('adam', 'mse')
icnn_f2.optimizer.learning_rate.assign(0.01)

t1 = now()
h = icnn_f2.fit(tf.concat([x,y], axis=-1), f2, epochs=EPOCHS, verbose=0)
t2 = now()
print(f'ICNN f2 took', t2 - t1, '(sec) to calibrate the model')
h_icnn_f2 = h.history['loss']

FFNN f1 took 0:00:34.678413 (sec) to calibrate the model
ICNN f1 took 0:00:30.773503 (sec) to calibrate the model
FFNN f2 took 0:00:28.291613 (sec) to calibrate the model
ICNN f2 took 0:00:35.666273 (sec) to calibrate the model


In [5]:
loss_hist = {'FFNN f1': h_ffnn_f1, 'ICNN f1': h_icnn_f1, 'FFNN f2': h_ffnn_f2, 'ICNN f2': h_icnn_f2}
p2_loss = plot_loss_history(loss_hist, *FIG_SIZE, use_latex_style=USE_LATEX_STYLE)
show(p2_loss)
save_figures_button([('2_2_loss', p2_loss),], svg_results_dir=PLOTS_DIR, png_results_dir=PLOTS_DIR)

ToggleButtons(description='Save Figures with format:', options=('svg', 'png', 'both'), tooltips=('Saves figure…

In [6]:
predictions1 = {
    'FFNN f1': ffnn_f1.predict(tf.concat([x,y], axis=-1)), 
    'ICNN f1': icnn_f1.predict(tf.concat([x,y], axis=-1)), 
}
p2_pred_f1 = plot_x_y_z(predictions1, x, y, f1, *FIG_SIZE, use_latex_style=USE_LATEX_STYLE)
show(p2_pred_f1)

save_figures_button([('2_2_prediction_f1', p2_pred_f1),], svg_results_dir=PLOTS_DIR, png_results_dir=PLOTS_DIR)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


ToggleButtons(description='Save Figures with format:', options=('svg', 'png', 'both'), tooltips=('Saves figure…

In [7]:
predictions2 = {
    'FFNN f2': icnn_f2.predict(tf.concat([x,y], axis=-1)), 
    'ICNN f2': ffnn_f2.predict(tf.concat([x,y], axis=-1))
}
p2_pred_f2 = plot_x_y_z(predictions2, x, y, f2, *FIG_SIZE, use_latex_style=USE_LATEX_STYLE)
show(p2_pred_f2)

save_figures_button([('2_2_prediction_f2', p2_pred_f2),], svg_results_dir=PLOTS_DIR, png_results_dir=PLOTS_DIR)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


ToggleButtons(description='Save Figures with format:', options=('svg', 'png', 'both'), tooltips=('Saves figure…

## 1.2.3 - Sobolev Training

In [8]:
def sobolev_loss(y_true, y_pred, lambda_output=1.0, lambda_derivative=0.1, num_outputs=1, num_gradients=2) -> tf.Tensor:
    y_true_output, y_true_grad = tf.split(y_true, num_or_size_splits=[num_outputs, num_gradients], axis=-1)
    y_pred_output, y_pred_grad = tf.split(y_pred, num_or_size_splits=[num_outputs, num_gradients], axis=-1)
    output_loss = tf.reduce_mean(tf.square(y_true_output - y_pred_output))
    gradient_loss = tf.reduce_mean(tf.square(y_true_grad - y_pred_grad))
    return lambda_output * output_loss + lambda_derivative * gradient_loss

In [9]:
EPOCHS = 500


# f2
icnn_f2 = ICNN(2, [32, 32, 32, 1], ['softplus', 'softplus', 'softplus', 'linear'])
icnn_f2.compile(optimizer=optimizers.Adam(learning_rate=0.01), loss='mse')
t1 = now()
h = icnn_f2.fit(tf.concat([x,y], axis=-1), f2, epochs=EPOCHS, verbose=0, batch_size=32)
t2 = now()
print(f'ICNN f2 took', t2 - t1, '(sec) to calibrate the model')
h_icnn_f2 = h.history['loss']


# # f2 and grads
icnn_f2_grad = ICNN(2, [32, 32, 32, 1], ['softplus', 'softplus', 'softplus', 'linear'], use_derivative=True)
icnn_f2_grad.compile(
    optimizer=optimizers.Adam(learning_rate=0.01), 
    loss=lambda true, pred: sobolev_loss(true, pred, lambda_derivative=0.1)
)
t1 = now()
h = icnn_f2_grad.fit(tf.concat([x,y], axis=-1), tf.concat([f2,del_f2__del_x], axis=-1), epochs=EPOCHS, verbose=0, batch_size=32)
t2 = now()
print(f'ICNN f2 and grads took', t2 - t1, '(sec) to calibrate the model')
h_icnn_f2_grad = h.history['loss']


# grads
icnn_grad = ICNN(2, [32, 32, 32, 1], ['softplus', 'softplus', 'softplus', 'linear'], use_derivative=True)
icnn_grad.compile(
    optimizer=optimizers.Adam(learning_rate=0.01), 
    loss=lambda true, pred: sobolev_loss(true, pred, lambda_output=0.0, lambda_derivative=1.0)
)
t1 = now()
h = icnn_grad.fit(tf.concat([x,y], axis=-1), tf.concat([f2,del_f2__del_x], axis=-1), epochs=EPOCHS, verbose=0, batch_size=32)
t2 = now()
print(f'FFNN f2 took', t2 - t1, '(sec) to calibrate the model')
h_icnn_grad = h.history['loss']

ICNN f2 took 0:00:30.661532 (sec) to calibrate the model
ICNN f2 and grads took 0:00:34.529994 (sec) to calibrate the model
FFNN f2 took 0:00:33.405741 (sec) to calibrate the model


In [10]:
loss_hist = {
    'f2 only': h_icnn_f2,
    'f2 and grad': h_icnn_f2_grad,
    'grad only': h_icnn_grad,
}
p3_loss_sob = plot_loss_history(loss_hist, *FIG_SIZE, use_latex_style=USE_LATEX_STYLE)
show(p3_loss_sob)

save_figures_button([('2_3_loss_sob', p3_loss_sob),], svg_results_dir=PLOTS_DIR, png_results_dir=PLOTS_DIR)

ToggleButtons(description='Save Figures with format:', options=('svg', 'png', 'both'), tooltips=('Saves figure…

### Show predictions

In [11]:
predictions = {
    'f2 only': (icnn_f2.predict(tf.concat([x,y], axis=-1))[:, 0]),
    'f2 and grad': (icnn_f2_grad.predict(tf.concat([x,y], axis=-1))[:, 0]),
}
p31_sob_f2 = plot_x_y_z(predictions, x, y, f2, *FIG_SIZE, use_latex_style=USE_LATEX_STYLE)
show(p31_sob_f2)

save_figures_button([('2_3_pred_sob_f2', p31_sob_f2),], svg_results_dir=PLOTS_DIR, png_results_dir=PLOTS_DIR)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


ToggleButtons(description='Save Figures with format:', options=('svg', 'png', 'both'), tooltips=('Saves figure…

In [12]:
predictions = {
    'grad only': (icnn_grad.predict(tf.concat([x,y], axis=-1))[:, 0]),
}
p32_sob_grad = plot_x_y_z(predictions, x, y, f2, *FIG_SIZE, use_latex_style=USE_LATEX_STYLE)
show(p32_sob_grad)

save_figures_button([('2_3_pred_sob_grad', p32_sob_grad),], svg_results_dir=PLOTS_DIR, png_results_dir=PLOTS_DIR)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


ToggleButtons(description='Save Figures with format:', options=('svg', 'png', 'both'), tooltips=('Saves figure…